## Importing Packages and Data

In [144]:
# Data manipulation
import pandas as pd
import numpy as np
import re

# API & data access
import json
import requests

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# Color mapping for folium
from matplotlib import cm
from matplotlib.colors import Normalize, to_hex

# Interactive plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [93]:

dataset_id = "d_8b84c4ee58e3cfc0ece0d773c8ca6abc"
base_url = "https://data.gov.sg/api/action/datastore_search"
all_records = []
start = 0
batch_size = 100

while True:
    url = f"{base_url}?resource_id={dataset_id}&limit={batch_size}&offset={start}"
    response = requests.get(url)
    if response.status_code != 200:
        print("Failed to fetch data.")
        break

    data = response.json()
    records = data["result"]["records"]
    
    if not records:
        break  # No more data to fetch

    all_records.extend(records)
    start += batch_size

df = pd.DataFrame(all_records)
print(df['town'].unique()) # To check whether all the towns are present


['ANG MO KIO' 'BEDOK' 'BISHAN' 'BUKIT BATOK' 'BUKIT MERAH' 'BUKIT PANJANG'
 'BUKIT TIMAH' 'CENTRAL AREA' 'CHOA CHU KANG' 'CLEMENTI' 'GEYLANG'
 'HOUGANG' 'JURONG EAST' 'JURONG WEST' 'KALLANG/WHAMPOA' 'MARINE PARADE'
 'PASIR RIS' 'PUNGGOL' 'QUEENSTOWN' 'SEMBAWANG' 'SENGKANG' 'SERANGOON'
 'TAMPINES' 'TOA PAYOH' 'WOODLANDS' 'YISHUN']


## Exploratory Data Analysis (EDA)

In [94]:
print(df.columns)
print(df.dtypes)
print(df.info())

Index(['_id', 'month', 'town', 'flat_type', 'block', 'street_name',
       'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price'],
      dtype='object')
_id                     int64
month                  object
town                   object
flat_type              object
block                  object
street_name            object
storey_range           object
floor_area_sqm         object
flat_model             object
lease_commence_date    object
remaining_lease        object
resale_price           object
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208758 entries, 0 to 208757
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   _id                  208758 non-null  int64 
 1   month                208758 non-null  object
 2   town                 208758 non-null  object
 3   flat_type            208758 non-null  object
 4  

In [95]:
df.head()

,_id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,1,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44,Improved,1979,61 years 04 months,232000
1,2,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67,New Generation,1978,60 years 07 months,250000
2,3,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,262000
3,4,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68,New Generation,1980,62 years 01 month,265000
4,5,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,265000


## Data Cleaning

In [96]:
# Convert column types
df['month'] = pd.to_datetime(df['month'], format='%Y-%m', errors='coerce')
df['floor_area_sqm'] = pd.to_numeric(df['floor_area_sqm'], errors='coerce')
df['resale_price'] = pd.to_numeric(df['resale_price'], errors='coerce')
df['lease_commence_date'] = pd.to_numeric(df['lease_commence_date'], errors='coerce')

In [97]:
# Check for missing values
print(df.isnull().sum())
# Drop rows with missing values
df.dropna(inplace=True)
# Check for duplicates
print(df.duplicated().sum())
# Drop duplicates
df.drop_duplicates(inplace=True)

_id                    0
month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
remaining_lease        0
resale_price           0
dtype: int64
0


## Questions of Contention

### Question 1: Which areas are high in demand in Singapore, based on resale prices?


Aim: To create a map of Singapore with gradients to indicate popularity

Plan:

- Obtain metric of comparison: Resale price per square metre (in general, encompassing all room types)

- Obtain geographical coordinates of Singapore's towns and map it to the current towns (estimated region will work)

- Obtain a Map plot that shows the different regions of Singapore and their resale price per square metre, with a heat gradient

In [138]:
# -------------------------------
# 1. Parse remaining lease into years
# -------------------------------
def parse_lease(lease_str):
    match = re.match(r"(\d+)\s+years\s+(\d+)\s+months", str(lease_str))
    if match:
        years = int(match.group(1))
        months = int(match.group(2))
        return years + months / 12
    return None

df["remaining_lease_years"] = df["remaining_lease"].apply(parse_lease)

# -------------------------------
# 2. Aggregate stats by town and flat_type
# -------------------------------
grouped = df.groupby(['town', 'flat_type'])

summary_df = grouped.agg(
    avg_resale_price=('resale_price', 'mean'),
    avg_floor_area=('floor_area_sqm', 'mean'),
    avg_transactions=('resale_price', 'count'),
    avg_remaining_lease=('remaining_lease_years', 'mean'),
    median_remaining_lease=('remaining_lease_years', 'median')
).reset_index()

# -------------------------------
# 3. Calculate avg_price_per_sqm
# -------------------------------
summary_df['avg_price_per_sqm'] = summary_df['avg_resale_price'] / summary_df['avg_floor_area']

# -------------------------------
# 4. Final tidy column order
# -------------------------------
summary_df = summary_df[[
    'town',
    'flat_type',
    'avg_resale_price',
    'avg_price_per_sqm',
    'avg_transactions',
    'avg_remaining_lease',
    'median_remaining_lease'
]]

# -------------------------------
# 5. Create flat type list per town (for lookup or mapping)
# -------------------------------
flat_types_by_town = (
    df.groupby('town')['flat_type']
    .unique()
    .apply(sorted)
    .reset_index()
    .rename(columns={'flat_type': 'available_flat_types'})
)

# Merge this into the summary for convenience (optional)
summary_df = pd.merge(summary_df, flat_types_by_town, on='town', how='left')

# -------------------------------
# 6. Preview result
# -------------------------------
print("Final Summary with Available Flat Types:")
print(summary_df.head())


Final Summary with Available Flat Types:
         town  flat_type  avg_resale_price  avg_price_per_sqm  \
0  ANG MO KIO     2 ROOM     253227.006369        5719.556898   
1  ANG MO KIO     3 ROOM     347882.618294        4899.831058   
2  ANG MO KIO     4 ROOM     547890.423423        5893.007461   
3  ANG MO KIO     5 ROOM     742301.653627        6245.922966   
4  ANG MO KIO  EXECUTIVE     924049.728000        5943.512195   

   avg_transactions  avg_remaining_lease  median_remaining_lease  \
0               157            57.958651               57.250000   
1              4548            59.255578               57.833333   
2              2442            67.212063               59.416667   
3              1271            71.562770               69.375000   
4               125            67.116987               70.833333   

                          available_flat_types  
0  [2 ROOM, 3 ROOM, 4 ROOM, 5 ROOM, EXECUTIVE]  
1  [2 ROOM, 3 ROOM, 4 ROOM, 5 ROOM, EXECUTIVE]  
2  [2 ROOM, 

In [132]:
town_coordinates = {
    'ANG MO KIO': (1.3691, 103.8454),
    'BEDOK': (1.3244, 103.9301),
    'BISHAN': (1.3509, 103.8485),
    'BUKIT BATOK': (1.3496, 103.7499),
    'BUKIT MERAH': (1.2777, 103.8203),
    'BUKIT PANJANG': (1.3784, 103.7633),
    'BUKIT TIMAH': (1.3294, 103.8021),
    'CENTRAL AREA': (1.2905, 103.8510),
    'CHOA CHU KANG': (1.3854, 103.7449),
    'CLEMENTI': (1.3151, 103.7655),
    'GEYLANG': (1.3160, 103.8855),
    'HOUGANG': (1.3711, 103.8922),
    'JURONG EAST': (1.3321, 103.7430),
    'JURONG WEST': (1.3400, 103.7064),
    'KALLANG/WHAMPOA': (1.3210, 103.8618),
    'MARINE PARADE': (1.3032, 103.9057),
    'PASIR RIS': (1.3730, 103.9497),
    'PUNGGOL': (1.4043, 103.9021),
    'QUEENSTOWN': (1.2944, 103.8031),
    'SEMBAWANG': (1.4491, 103.8205),
    'SENGKANG': (1.3911, 103.8950),
    'SERANGOON': (1.3496, 103.8739),
    'TAMPINES': (1.3496, 103.9568),
    'TOA PAYOH': (1.3343, 103.8518),
    'WOODLANDS': (1.4360, 103.7865),
    'YISHUN': (1.4294, 103.8350)
}


In [142]:
# -----------------------------
# 1. Map coordinates to towns
# -----------------------------
summary_df["lat"] = summary_df["town"].map(lambda t: town_coordinates.get(t, (None, None))[0])
summary_df["lon"] = summary_df["town"].map(lambda t: town_coordinates.get(t, (None, None))[1])

# -----------------------------
# 2. Identify towns missing coordinates
# -----------------------------
missing = summary_df.loc[
    summary_df["lat"].isna() | summary_df["lon"].isna(), "town"
].unique()

if len(missing) > 0:
    print("Missing coordinates for the following towns (not shown on map):")
    for t in missing:
        print(" -", t)

# -----------------------------
# 3. Drop rows with missing coordinates and reset index
# -----------------------------
summary_df = summary_df.dropna(subset=["lat", "lon"]).reset_index(drop=True)

# -----------------------------
# 4. Scale circle radius by transaction count
# -----------------------------
max_tx = summary_df["avg_transactions"].max()
min_radius = 5    # Minimum marker size
max_radius = 25   # Maximum marker size

# -----------------------------
# 5. Color map normalization based on price per sqm
# -----------------------------
norm = Normalize(
    vmin=summary_df["avg_price_per_sqm"].min(),
    vmax=summary_df["avg_price_per_sqm"].max()
)
colors = [to_hex(cm.YlOrRd(norm(v))) for v in summary_df["avg_price_per_sqm"]]

# -----------------------------
# 6. Create base map centered on Singapore
# -----------------------------
m = folium.Map(location=[1.3521, 103.8198], zoom_start=11)

# -----------------------------
# 7. Add circle markers with detailed HTML popups
# -----------------------------
for idx, row in summary_df.iterrows():
    # Compute proportional radius
    radius = min_radius + (row["avg_transactions"] / max_tx) * (max_radius - min_radius)

    # Format HTML popup content
    html_popup = folium.Popup(
        f"<b>{row['town']}</b><br>"
        f"Avg Resale Price: {row['avg_price_per_sqm']:.2f} SGD/sqm<br>"
        f"Avg Transactions: {int(row['avg_transactions'])}<br>"
        f"Avg Remaining Lease: {row['avg_remaining_lease']:.1f} years<br>"
        f"Median Remaining Lease: {row['median_remaining_lease']:.1f} years <br>"
        f"Room Types: {row['available_flat_types']}<br>",
        max_width=250
    )

    # Add circle marker to the map
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=radius,
        popup=html_popup,
        color=colors[idx],
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# -----------------------------
# 8. Save map to HTML file
# -----------------------------
m.save("hdb_demand_map.html")

# Optional: if in a Jupyter notebook, display the map inline
m


#### Findings

According to the Law of Demand, 

### Question 2: How has housing prices changed over time for the same housing type and area?

Aims:

- To prepare data for to visualise how prices have changed. 

- Can go from Town, to Flat Type within the town

- Point out some anomalies too

In [147]:

# -------------------------------
# 1. Group by flat_type and month, compute median resale price per sqm
# -------------------------------
room_median_df = df.groupby(['flat_type', 'month']).agg(
    median_price_per_sqm=('resale_price', lambda x: x.median() / df['floor_area_sqm'].median())
).reset_index()

# -------------------------------
# 2. Convert 'month' to datetime
# -------------------------------
room_median_df['month'] = pd.to_datetime(room_median_df['month'], format='%Y-%m', errors='coerce')

# -------------------------------
# 3. Drop any invalid or missing rows
# -------------------------------
room_median_df = room_median_df.dropna(subset=['month', 'median_price_per_sqm'])

# -------------------------------
# 4. Create interactive line chart using Plotly
# -------------------------------
fig = px.line(
    room_median_df,
    x='month',
    y='median_price_per_sqm',
    color='flat_type',
    markers=True,
    title='Median Resale Price per sqm Over Time (Aggregated Across Towns)',
    labels={
        'month': 'Year',
        'median_price_per_sqm': 'Median Price per sqm (SGD)',
        'flat_type': 'Flat Type'
    }
)

# Customize layout
fig.update_layout(
    hovermode='x unified',
    template='plotly_white',
    legend_title_text='Flat Type'
)

# -------------------------------
# 5. Show interactive chart in notebook or browser
# -------------------------------
fig.show()


In [ ]:


# -------------------------------
# 1. Group by town, flat type, and month
# -------------------------------
room_price_df = df.groupby(['town', 'flat_type', 'month']).agg(
    avg_price_per_sqm=('resale_price', lambda x: x.mean() / df['floor_area_sqm'].mean()),
    avg_transactions=('resale_price', 'count'),
    avg_remaining_lease=('lease_commence_date', lambda x: 2023 - x.mean()),
    median_remaining_lease=('lease_commence_date', lambda x: 2023 - x.median())
).reset_index()

# -------------------------------
# 2. Pivot and melt to long format
# -------------------------------
room_price_pivot = room_price_df.pivot_table(
    index=['town', 'flat_type'],
    columns='month',
    values='avg_price_per_sqm',
    fill_value=0
).reset_index()

room_price_long = room_price_pivot.melt(
    id_vars=['town', 'flat_type'],
    var_name='month',
    value_name='avg_price_per_sqm'
)

# -------------------------------
# 3. Convert 'month' to datetime
# -------------------------------
room_price_long['month'] = pd.to_datetime(room_price_long['month'], format='%Y-%m', errors='coerce')

# -------------------------------
# 4. Drop rows with no transactions
# -------------------------------
room_price_long = room_price_long[room_price_long['avg_price_per_sqm'] > 0]

# -------------------------------
# 5. Set up towns and flat types
# -------------------------------
towns = sorted(room_price_long['town'].unique())
flat_types = sorted(room_price_long['flat_type'].unique())

# Define consistent colors for each flat type
flat_type_colors = {
    "1 ROOM": "#1f77b4",       # Blue
    "2 ROOM": "#636EFA",       # Indigo
    "3 ROOM": "#EF553B",       # Red
    "4 ROOM": "#00CC96",       # Green
    "5 ROOM": "#AB63FA",       # Purple
    "EXECUTIVE": "#FFA15A",    # Orange
    "MULTI-GENERATION": "#19D3F3",  # Cyan
}


flat_types = sorted(room_price_long['flat_type'].unique())
color_palette = px.colors.qualitative.Set2  # or 'Plotly', 'Dark24', etc.

# Map flat types to a color (repeating if needed)
flat_type_colors = {ft: color_palette[i % len(color_palette)] for i, ft in enumerate(flat_types)}


# Split towns into 3 groups
n_groups = 3
batches = [towns[i::n_groups] for i in range(n_groups)]

# -------------------------------
# 6. Loop through each batch of towns to create subplots
# -------------------------------
for batch_num, town_batch in enumerate(batches, start=1):
    n_cols = 3
    n_rows = -(-len(town_batch) // n_cols)  # Ceiling division

    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        subplot_titles=town_batch,
        shared_xaxes=True,
        shared_yaxes=True,
        horizontal_spacing=0.05,
        vertical_spacing=0.1
    )

    row, col = 1, 1
    shown_flat_types = set()

    for town in town_batch:
        town_data = room_price_long[room_price_long['town'] == town]

        for flat_type in flat_types:
            flat_data = town_data[town_data['flat_type'] == flat_type]
            if flat_data.empty:
                continue

            fig.add_trace(
                go.Scatter(
                    x=flat_data['month'],
                    y=flat_data['avg_price_per_sqm'],
                    mode='lines+markers',
                    name=flat_type,
                    legendgroup=flat_type,
                    showlegend=(flat_type not in shown_flat_types),
                    line=dict(color=flat_type_colors[flat_type])
                ),
                row=row,
                col=col
            )
            shown_flat_types.add(flat_type)

        # Advance subplot position
        if col < n_cols:
            col += 1
        else:
            col = 1
            row += 1

    # -------------------------------
    # 7. Update layout and show
    # -------------------------------
    fig.update_layout(
        height=n_rows * 350,
        width=1200,
        title_text=f"HDB Resale Price Trends per sqm — Town Group {batch_num}",
        showlegend=True,
        template="plotly_white"
    )

    fig.update_xaxes(title_text="Month", tickangle=45)
    fig.update_yaxes(title_text="Avg Price per sqm (SGD)")

    fig.show()
    # Optional: Save to HTML
    # fig.write_html(f"price_trends_group_{batch_num}.html")
